In [ ]:
from glob import glob 
import pandas as pd

ix_sample = -3
sample_paths = glob('../data/*/*.csv')
[print(x) for x in sample_paths]
sample_path = sample_paths[ix_sample]
print('usamos este:',sample_path)
sample = pd.read_csv(sample_path, index_col=0)

actions = sample[sample['event'].map(lambda x: 'pressed' in x or 'scroll' in x)].copy()
actions['event'] = actions['event'].map(lambda x: x.replace('pressed ',''))
vocab = set(actions['event'].unique())

keys = []
unknowns = []
chars = []
others = []
for x in vocab:
    if 'Key.' in x:
        keys.append(x)
    elif '<' in x and '>' in x:
        unknowns.append(x)
    elif len(x) == 1: 
        chars.append(x)
    else: 
        others.append(x)

print()
keys.sort()
print(len(keys), keys)
print()
unknowns.sort()
print(len(unknowns), unknowns) 
print()
chars.sort()
print(len(chars), chars) 
print()
others.sort()
print(len(others), others) 
print()
print(len(keys)+ len(unknowns) + len(chars), '=', len(vocab))

In [6]:
import time 
from pynput.keyboard import Key
from pynput.keyboard import Controller as kController
from pynput.mouse import Button 
from pynput.mouse import Controller as mController


class pcController(): 
    def __init__(self):
        self.mouse = mController()
        self.keyboard = kController()
        self.mouse_mapping = {
                'Button.left': lambda position, endposition: self.clickleft(position),
                'Button.left.double': lambda position, endposition: self.doubleclickleft(position),
                'Button.left.drag': lambda position, endposition: self.leftdrag(position, endposition), 
                'Button.right': lambda position, endposition: self.clickright(position),
                'Scroll.down': lambda position, endposition: self.scrolldown(position),
                'Scroll.up': lambda position, endposition: self.scrollup(position),
            }
        self.none_mouse = lambda position, endposition: print("invalid mouse action")
        self.umapping = {
                '<269025048>': ['f4'], 
                '<269025062>': ['f6'],  
                '<65027>': ['alt_gr'], 
                '<65056>': ['shift_r', 'tab'],
                "['´']": [],   # tilde
                "['¨']": [],   # tilde con shift_r
                '"\'"': ["'"],   
            }

    def run(self, action, position, endposition, delay): 
        # Mouse
        if "Button." in action or "Scroll." in action:
            self.mouse_mapping.get(action, self.none_mouse)(position, endposition)
        # Writing
        elif len(action) == 1:
            self.push(action)
        # Known Keys
        elif "Key." in action:
            keyname = action.replace('Key.','')
            if keyname in Key.__dict__.keys():    
                button = Key.__dict__[keyname]
                self.push(button)
            else:
                print("invalid key action", action)
        # Unknown keys 
        elif action in self.umapping.keys():      
            key_sequence = self.umapping[action]
            if len(key_sequence) == 1: 
                if key_sequence[0] == "'":  
                    self.push("'")
                else: 
                    button = Key.__dict__[key_sequence[0]]
                    self.push(button)
            elif len(key_sequence) == 2: 
                button1 = Key.__dict__[key_sequence[0]]
                button2 = Key.__dict__[key_sequence[1]]
                with self.keyboard.pressed(button1):
                    self.push(button2)           
        else:
            print(f'The action {action} is not a key, not a mouse, not a char')
        time.sleep(delay)

    def push(self, button):
        self.keyboard.press(button)
        self.keyboard.release(button)

    def clickleft(self, position):
        self.mouse.position = position
        self.mouse.click(Button.left, 1)

    def doubleclickleft(self, position):
        self.mouse.position = position
        self.mouse.click(Button.left, 2)

    def leftdrag(self, position, endposition):
        self.mouse.position = position
        self.mouse.press(Button.left)
        self.mouse.position = endposition
        self.mouse.release(Button.left)

    def clickright(self, position):
        self.mouse.position = position
        self.mouse.click(Button.right, 1)

    def scrollup(self, position):
        self.mouse.position = position
        self.mouse.scroll(0, -1)

    def scrolldown(self, position):
        self.mouse.position = position
        self.mouse.scroll(0, 1)

"""
endposition = [500,500]
position = [1000,50]
action = 'Scroll.up' #'Button.right'
delay = 0.5
pcontroller(action, position, endposition, delay) 
"""
pccontroller = pcController()
endposition = []
position = [1000,750]
delay = 0.1
pccontroller.clickleft(position)
for action in unknowns:
    pccontroller.run(action, position, endposition, delay) 


In [ ]:












"#$%&()*+,-./0123456789:;<=>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz{|}¡°¿Ññ"

''




"#$%&()*+,-./0123456789:;<=>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz{|}¡°¿Ññ"

In [ ]:
def clickleft(position):
    mouse.position = position
    mouse.click(Button.left, 1)

def doubleclickleft(position):
    mouse.position = position
    mouse.click(Button.left, 2)

def leftdrag(position, endposition):
    mouse.position = position
    mouse.press(Button.left)
    mouse.position = endposition
    mouse.release(Button.left)

def clickright(position):
    mouse.position = position
    mouse.click(Button.right, 1)

def scrollup(position):
    mouse.position = position
    mouse.scroll(0, -1)

def scrolldown(position):
    mouse.position = position
    mouse.scroll(0, 1)

mouse_mapping = {
    'Button.left': lambda position, endposition: clickleft(position),
    'Button.left.double': lambda position, endposition: doubleclickleft(position),
    'Button.left.drag': lambda position, endposition: leftdrag(position, endposition), 
    'Button.right': lambda position, endposition: clickright(position),
    'Scroll.down': lambda position, endposition: scrolldown(position),
    'Scroll.up': lambda position, endposition: scrollup(position),
}
none_mouse_mapping = lambda position, endposition: print("invalid mouse action")

endposition = [500,500]
position = [1000,500]
action = 'Scroll.up'
mouse_mapping.get(action, none_mouse_mapping)(position, endposition)

In [ ]:
# mapping from pynput to pyautogui 
key_mapping = {
    'Key.alt': ['alt'], 
    'Key.backspace': ['backspace'], 
    'Key.caps_lock': ['capslock'], 
    'Key.cmd': ['command'], 
    'Key.ctrl': ['ctrl'], 
    'Key.ctrl_r': ['ctrlright'], 
    'Key.delete': ['delete'], 
    'Key.down': ['down'], 
    'Key.enter': ['enter'], 
    'Key.esc': ['esc'], 
    'Key.left': ['left'], 
    'Key.media_next': ['nexttrack'], 
    'Key.media_play_pause': ['playpause'], 
    'Key.media_previous': ['prevtrack'], 
    'Key.media_volume_down': ['volumedown'], 
    'Key.media_volume_mute': ['volumemute'], 
    'Key.media_volume_up': ['volumeup'],
    'Key.menu': ['f5'],   #No equivalente
    'Key.right': ['right'], 
    'Key.shift': ['shift'], 
    'Key.shift_r': ['shiftright'], 
    'Key.space': ['space'], 
    'Key.tab': ['tab'], 
    'Key.up': ['up'],

    '<269025048>': ['f4'], 
    '<269025062>': ['f6'],  
    '<65027>': ['altright'], 
    '<65056>': ['shift_r', 'tab'],

    "['´']": [],   # tilde
    "['¨']": [],   # tilde con shift_r
    '"\'"': ["'"], 
    }

DEcision 
Cambia a usar solo pynput porque evito el mapeo entre pynput y pyautogui 
Además, pyautogui no tiene algunos simbolos. 
len('!"#$%&()*+,-./0123456789:;<=>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz{|}¡°¿Ññ')
len('!"#$%&()*+,-./0123456789:;<=>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz{|}¡°¿Ññ')  # keyboard.press(action)  keyboard.release(action)
len('!"#$%&()**,-.70123456789:,>0>?ABCDEFGHIJKLMNOPQRSTUVWXYZ{}_abcdefghijklmnopqrstuvwxyz[°]"')   # pyautogui.write(action)

In [ ]:
import pyautogui

mouse_mapping = {
    'Button.left': lambda position, endposition: pyautogui.click(x=position[0], y=position[1]),
    'Button.left.double': lambda position, endposition: pyautogui.doubleClick(x=position[0],y=position[1]),
    'Button.left.drag': lambda position, endposition: leftdrag(position, endposition), 
    'Button.right': lambda position, endposition: pyautogui.click(button='right', x=position[0], y=position[1]),
    'scroll.down': lambda position, endposition: pyautogui.scroll(-1, x=position[0], y=position[1]),
    'scroll.up': lambda position, endposition: pyautogui.scroll(1, x=position[0], y=position[1])
}
none_mouse_mapping = lambda position, endposition: print("invalid mouse action")

endposition = [500,500]
position = [1000,50]
action = 'Button.left'
mouse_mapping.get(action, none_mouse_mapping)(position, endposition)